<a href="https://colab.research.google.com/github/vinayk19/Assignment/blob/master/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

In [0]:
train_data = datasets.MNIST(root = "MNIST/processed/training.pt", train = True, target_transform= None, download= True)
test_data = datasets.MNIST(root = "MNIST/processed/test.pt", train = False, target_transform= None, download = True)

In [0]:
tain_loader = DataLoader(dataset = train_data, batch_size= 64, shuffle= True)
test_loader = DataLoader(dataset = test_data, batch_size= 64, shuffle= False)

In [0]:
class mnistmodel(torch.nn.Module):
  def __init__(self):
    super(mnistmodel, self).__init__()
    self.conv1 = torch.nn.Conv2d(1, 10 , kernel_size= 5, stride= 1, padding= 0)
    self.conv2 = torch.nn.Conv2d(10, 20, kernel_size= 5, stride= 1, padding= 0)
    self.maxpool = torch.nn.MaxPool2d(kernel_size=2)
    self.linear = torch.nn.Linear(320,10)

  def forward(self, data):
    in_size = x.size(0)
    x = torch.nn.ReLU(self.maxpool(self.conv1(data)))
    x = torch.nn.ReLU(self.maxpool(self.conv2(x)))  
    x = torch.view(in_size, -1)
    x = self.linear(x)
    out = torch.nn.functional.Softmax(x, dim=0)
    return out